# Benchmark Analysis: PyPSA vs Antares

This notebook provides a function to analyze benchmark results for a specific study by row number.


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path
import seaborn as sns

# Set plotting style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (14, 8)
plt.rcParams['font.size'] = 10

# Get the project root directory
def get_results_path():
    """Get the path to the benchmark results CSV file."""
    current_dir = Path().resolve()
    # Try to find the project root
    for parent in current_dir.parents:
        if (parent / "tmp" / "benchmark_results" / "all_studies_results.csv").exists():
            return parent / "tmp" / "benchmark_results" / "all_studies_results.csv"
    # Fallback: assume we're in project root
    return Path("tmp") / "benchmark_results" / "all_studies_results.csv"


: 

In [ ]:
def analyze_benchmark_study(row_number: int, results_file: Path = None):
    """
    Analyze and plot benchmark results for a specific study.
    
    Parameters:
    -----------
    row_number : int
        Row number (0-indexed) of the study to analyze
    results_file : Path, optional
        Path to the results CSV file. If None, will try to find it automatically.
    """
    # Load data
    if results_file is None:
        results_file = get_results_path()
    
    if not results_file.exists():
        raise FileNotFoundError(f"Results file not found: {results_file}")
    
    df_all = pd.read_csv(results_file)
    
    if row_number < 0 or row_number >= len(df_all):
        raise ValueError(f"Row number must be between 0 and {len(df_all)-1}. "
                        f"Total studies available: {len(df_all)}")
    
    df = df_all.iloc[[row_number]].copy()
    row = df.iloc[0]
    
    # Print overview statistics
    print("=" * 80)
    print(f"BENCHMARK ANALYSIS - STUDY ROW {row_number}")
    print("=" * 80)
    
    print(f"\n📊 NETWORK INFORMATION:")
    print(f"  Network Name: {row['pypsa_network_name']}")
    print(f"  Number of Time Steps: {int(row['number_of_time_steps'])}")
    print(f"  PyPSA Version: {row['pypsa_version']}")
    print(f"  Antares Version: {row['antares_version']}")
    
    print(f"\n🔧 NETWORK COMPONENTS:")
    print(f"  Buses: {int(row['number_of_buses'])}")
    print(f"  Generators: {int(row['number_of_generators'])}")
    print(f"  Loads: {int(row['number_of_loads'])}")
    print(f"  Links: {int(row['number_of_links'])}")
    print(f"  Storage Units: {int(row['number_of_storage_units'])}")
    print(f"  Stores: {int(row['number_of_stores'])}")
    print(f"  Lines: {int(row['number_of_lines'])}")
    print(f"  Transformers: {int(row['number_of_transformers'])}")
    print(f"  Shunt Impedances: {int(row['number_of_shunt_impedances'])}")
    
    print(f"\n⏱️  TIMING INFORMATION:")
    print(f"  Parsing Time: {row['parsing_time']:.4f} s")
    print(f"  Preprocessing Time (PyPSA): {row['preprocessing_time_pypsa_network']:.4f} s")
    print(f"  PyPSA to GEMS Conversion Time: {row['pypsa_to_gems_conversion_time']:.4f} s")
    print(f"  Build Optimization Problem Time (PyPSA): {row['build_optimization_problem_time_pypsa']:.4f} s")
    print(f"  PyPSA Optimization Time: {row['pypsa_optimization_time']:.4f} s")
    print(f"  PyPSA Total Time: {row['total_time_pypsa']:.4f} s")
    print(f"  Antares Modeler Total Time: {row['antares_modeler__total_time']:.4f} s")
    
    print(f"\n📈 OPTIMIZATION PROBLEM SIZE:")
    print(f"  PyPSA Constraints: {int(row['number_of_constraints_pypsa'])}")
    print(f"  Antares Constraints: {int(row['number_of_constraints_antares'])}")
    print(f"  Constraints Ratio (PyPSA/Antares): {row['number_of_constraints_pypsa'] / row['number_of_constraints_antares']:.4f}")
    print(f"  PyPSA Variables: {int(row['number_of_variables_pypsa'])}")
    print(f"  Antares Variables: {int(row['number_of_variables_antares'])}")
    print(f"  Variables Ratio (PyPSA/Antares): {row['number_of_variables_pypsa'] / row['number_of_variables_antares']:.4f}")
    
    print(f"\n🎯 OBJECTIVE VALUES:")
    print(f"  PyPSA Objective: {row['pypsa_objective']:.6f}")
    print(f"  Antares Objective: {row['antares_objective_value']:.6f}")
    obj_diff = row['pypsa_objective'] - row['antares_objective_value']
    obj_diff_pct = (obj_diff / row['antares_objective_value']) * 100
    print(f"  Difference: {obj_diff:.6f} ({obj_diff_pct:+.4f}%)")
    
    print(f"\n⚙️  SOLVER INFORMATION:")
    print(f"  PyPSA Solver: {row['solver_name_pypsa']} {row['solver_version_pypsa']}")
    print(f"  Antares Solver: {row['antares_solver_name']}")
    print(f"  Antares Solver Parameters: {row['antares_solver_parameters']}")
    
    print(f"\n📊 PERFORMANCE COMPARISON:")
    time_ratio = row['total_time_pypsa'] / row['antares_modeler__total_time']
    print(f"  Time Ratio (PyPSA/Antares): {time_ratio:.4f}x")
    if time_ratio < 1:
        print(f"  → PyPSA is {1/time_ratio:.2f}x faster")
    else:
        print(f"  → Antares is {time_ratio:.2f}x faster")
    
    print("\n" + "=" * 80)
    
    # Create visualizations
    fig = plt.figure(figsize=(16, 12))
    
    # 1. Objective Value Comparison
    ax1 = plt.subplot(2, 3, 1)
    categories = ['PyPSA', 'Antares']
    objectives = [row['pypsa_objective'], row['antares_objective_value']]
    bars = ax1.bar(categories, objectives, color=['steelblue', 'coral'], alpha=0.7, edgecolor='black')
    ax1.set_ylabel('Objective Value', fontsize=11)
    ax1.set_title('Objective Value Comparison', fontsize=12, fontweight='bold')
    ax1.grid(True, alpha=0.3, axis='y')
    # Add value labels on bars
    for bar, val in zip(bars, objectives):
        height = bar.get_height()
        ax1.text(bar.get_x() + bar.get_width()/2., height,
                f'{val:.2e}', ha='center', va='bottom', fontsize=9)
    
    # 2. Time Comparison
    ax2 = plt.subplot(2, 3, 2)
    times = [row['total_time_pypsa'], row['antares_modeler__total_time']]
    bars = ax2.bar(categories, times, color=['steelblue', 'coral'], alpha=0.7, edgecolor='black')
    ax2.set_ylabel('Time (seconds)', fontsize=11)
    ax2.set_title('Total Time Comparison', fontsize=12, fontweight='bold')
    ax2.grid(True, alpha=0.3, axis='y')
    for bar, val in zip(bars, times):
        height = bar.get_height()
        ax2.text(bar.get_x() + bar.get_width()/2., height,
                f'{val:.3f}s', ha='center', va='bottom', fontsize=9)
    
    # 3. Constraints Comparison
    ax3 = plt.subplot(2, 3, 3)
    constraints = [int(row['number_of_constraints_pypsa']), int(row['number_of_constraints_antares'])]
    bars = ax3.bar(categories, constraints, color=['steelblue', 'coral'], alpha=0.7, edgecolor='black')
    ax3.set_ylabel('Number of Constraints', fontsize=11)
    ax3.set_title('Constraints Comparison', fontsize=12, fontweight='bold')
    ax3.grid(True, alpha=0.3, axis='y')
    for bar, val in zip(bars, constraints):
        height = bar.get_height()
        ax3.text(bar.get_x() + bar.get_width()/2., height,
                f'{val:,}', ha='center', va='bottom', fontsize=9)
    
    # 4. Variables Comparison
    ax4 = plt.subplot(2, 3, 4)
    variables = [int(row['number_of_variables_pypsa']), int(row['number_of_variables_antares'])]
    bars = ax4.bar(categories, variables, color=['steelblue', 'coral'], alpha=0.7, edgecolor='black')
    ax4.set_ylabel('Number of Variables', fontsize=11)
    ax4.set_title('Variables Comparison', fontsize=12, fontweight='bold')
    ax4.grid(True, alpha=0.3, axis='y')
    for bar, val in zip(bars, variables):
        height = bar.get_height()
        ax4.text(bar.get_x() + bar.get_width()/2., height,
                f'{val:,}', ha='center', va='bottom', fontsize=9)
    
    # 5. Time Breakdown (PyPSA)
    ax5 = plt.subplot(2, 3, 5)
    pypsa_times = {
        'Preprocessing': row['preprocessing_time_pypsa_network'],
        'Conversion': row['pypsa_to_gems_conversion_time'],
        'Build Model': row['build_optimization_problem_time_pypsa'],
        'Optimization': row['pypsa_optimization_time']
    }
    ax5.pie(pypsa_times.values(), labels=pypsa_times.keys(), autopct='%1.1f%%',
            startangle=90, colors=plt.cm.Set3.colors)
    ax5.set_title('PyPSA Time Breakdown', fontsize=12, fontweight='bold')
    
    # 6. Objective Difference
    ax6 = plt.subplot(2, 3, 6)
    diff_pct = obj_diff_pct
    colors_bar = ['green' if abs(diff_pct) < 0.01 else 'orange' if abs(diff_pct) < 1 else 'red']
    bars = ax6.bar(['Objective\nDifference'], [diff_pct], color=colors_bar, alpha=0.7, edgecolor='black')
    ax6.axhline(y=0, color='black', linestyle='-', linewidth=1)
    ax6.set_ylabel('Relative Difference (%)', fontsize=11)
    ax6.set_title(f'Objective Difference\n(PyPSA - Antares) / Antares × 100%', fontsize=12, fontweight='bold')
    ax6.grid(True, alpha=0.3, axis='y')
    for bar, val in zip(bars, [diff_pct]):
        height = bar.get_height()
        ax6.text(bar.get_x() + bar.get_width()/2., height,
                f'{val:+.4f}%', ha='center', va='bottom' if val >= 0 else 'top', fontsize=10, fontweight='bold')
    
    plt.suptitle(f'Benchmark Analysis - Study Row {row_number}: {row["pypsa_network_name"]}', 
                fontsize=14, fontweight='bold', y=0.995)
    plt.tight_layout(rect=[0, 0, 1, 0.99])
    plt.show()
    
    return df


## Usage

Call the function with a row number (0-indexed) to analyze that study:


In [ ]:
# Example: Analyze study at row 0
analyze_benchmark_study(row_number=0)


In [ ]:
# Example: Analyze study at row 1
analyze_benchmark_study(row_number=1)


## List All Available Studies


In [ ]:
# Show all available studies
results_file = get_results_path()
df_all = pd.read_csv(results_file)

print(f"Total studies available: {len(df_all)}")
print("\nAvailable studies:")
print("-" * 80)
for idx in range(len(df_all)):
    row = df_all.iloc[idx]
    print(f"Row {idx:2d}: {row['pypsa_network_name']:30s} | "
          f"Time Steps: {int(row['number_of_time_steps']):5d} | "
          f"PyPSA Time: {row['total_time_pypsa']:7.3f}s | "
          f"Antares Time: {row['antares_modeler__total_time']:7.3f}s")
